<a href="https://colab.research.google.com/github/eoleedi/GenAI-2025/blob/main/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

主題二

## 1. GAN（生成對抗網絡）背後原理
GAN（Generative Adversarial Network）是一種生成模型，由 Goodfellow 等人於 2014 年提出。GAN 由兩個神經網絡組成：
- **生成器（Generator, G）**：負責從隨機噪聲生成逼真的數據。
- **判別器（Discriminator, D）**：負責判斷輸入數據是真實的還是來自生成器。

這兩個網絡透過對抗學習進行訓練，生成器試圖騙過判別器，而判別器則試圖區分真實數據與生成數據。這種對抗的過程促使生成器產生更高品質的數據。
## 2. Cross Entropy

Cross Entropy (CE) 用來衡量兩個機率分佈之間的差異，特別適用於分類問題。其公式如下：


$$H(p, q) = - \sum_{i} p(x_i) \log q(x_i)$$

其中：
- $ p(x) $ 是真實分佈（通常是 One-hot 標籤）。
- $ q(x) $ 是模型預測的機率分佈。

當 $ q(x) $ 與 $ p(x) $ 越接近時，Cross Entropy值越小，表示模型的預測與真實標籤越接近。


In [6]:

import numpy as np

def cross_entropy(p, q):
    return -np.sum(p * np.log(q))

p = np.array([1, 0, 0])  # 真實標籤（One-hot）
q1 = np.array([0.7, 0.2, 0.1])  # 預測較準確
q2 = np.array([0.4, 0.4, 0.2])  # 預測較不準確

print("Cross Entropy (q1):", cross_entropy(p, q1))
print("Cross Entropy (q2):", cross_entropy(p, q2))


Cross Entropy (q1): 0.35667494393873245
Cross Entropy (q2): 0.916290731874155



## 3. KL Divergence（Kullback-Leibler 散度）

KL 散度衡量兩個機率分佈的相對熵，表示 $ q(x) $ 與 $ p(x) $ 之間的信息損失：


$$D_{KL}(p || q) = \sum_{i} p(x_i) \log \frac{p(x_i)}{q(x_i)}$$


- 如果 $ p(x) $ 和 $ q(x) $ 完全相同，則 KL 散度為 0。
- KL 散度非對稱，即 $ D_{KL}(p || q) \neq D_{KL}(q || p) $。

In [5]:

def kl_divergence(p, q, epsilon=1e-10):
    p = np.array(p, dtype=np.float64)
    q = np.array(q, dtype=np.float64)

    # Apply additive smoothing
    p = (p + epsilon) / np.sum(p + epsilon)
    q = (q + epsilon) / np.sum(q + epsilon)

    # Compute KL divergence
    return np.sum(p * np.log(p / q))

print("KL Divergence (q1):", kl_divergence(p, q1))
print("KL Divergence (q2):", kl_divergence(p, q2))

KL Divergence (q1): 0.35667493961057245
KL Divergence (q2): 0.9162907271882996




## 4. Cross Entropy 與 KL Divergence 的關係

Cross Entropy 與 KL 散度的關係如下：

$$
H(p, q) = D_{KL}(p || q) + H(p)
$$

其中 $ H(p) $ 為 $ p(x) $ 本身的熵，表示真實分佈的固有不確定性。由於 $ H(p) $ 不受 $ q(x) $ 影響，最小化Cross Entropy等價於最小化 KL 散度。

## 5. GAN 訓練中的Cross Entropy與 KL 散度

在 GAN 的訓練中，判別器的目標是最大化分類真實與偽造樣本的準確率，而生成器的目標是最小化這個判別準確率（讓偽造樣本更像真實樣本）。

- 判別器損失：
  $$
  L_D = - \mathbb{E}_{x \sim p_{data}} [\log D(x)] - \mathbb{E}_{z \sim p_z} [\log (1 - D(G(z)))]
  $$
- 生成器損失：
  $$
  L_G = - \mathbb{E}_{z \sim p_z} [\log D(G(z))]
  $$

其中 $ D(x) $ 表示判別器對真實樣本的輸出機率，$ G(z) $ 表示生成器產生的樣本。

這些損失函數的設計本質上是基於Cross Entropy，使得 GAN 訓練的過程類似於一個二元分類問題。

## 6. 程式實驗：GAN 訓練示例

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

discriminator = nn.Sequential(
    nn.Linear(2, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

generator = nn.Sequential(
    nn.Linear(2, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

criterion = nn.BCELoss()  # Cross Entropy損失
opt_d = optim.Adam(discriminator.parameters(), lr=0.001)
opt_g = optim.Adam(generator.parameters(), lr=0.001)

real_data = torch.ones(10, 2)  # 假設真實數據
fake_data = generator(torch.randn(10, 2))

d_real = discriminator(real_data)
d_fake = discriminator(fake_data)

loss_d = criterion(d_real, torch.ones_like(d_real)) + criterion(d_fake, torch.zeros_like(d_fake))
loss_g = criterion(d_fake, torch.ones_like(d_fake))

print("Discriminator Loss:", loss_d.item())
print("Generator Loss:", loss_g.item())


Discriminator Loss: 1.5187180042266846
Generator Loss: 0.6190792918205261



## 7. 使用情境與比較

| 指標 | Cross Entropy | KL 散度 (KL Divergence) |
|------|------------------|------------------|
| 主要用途 | 衡量分類模型的預測準確度 | 衡量機率分佈的相似度 |
| 計算方式 | 直接計算兩個分佈的對數機率 | 需要計算兩個分佈的相對熵 |
| 在 GAN 的應用 | 作為損失函數來訓練判別器與生成器 | 在某些變分推理方法中使用 |

## 8. 結論

Cross Entropy和 KL 散度在機器學習中的應用非常廣泛。在 GAN 訓練中，Cross Entropy作為損失函數驅動著生成器與判別器的對抗學習，而 KL 散度則在變分自編碼器（VAE）等模型中扮演重要角色。透過數學推導和程式實驗，我們可以直觀理解這兩者的關係，並選擇適合的損失函數來優化模型。
